In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score, precision_score
import wandb
import xgboost as xgb

In [1]:
import os
import torch
import numpy as np
from tqdm import tqdm
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import Blip2Processor, Blip2Model

# Set device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

# Load BLIP-2 model and processor
model_id = "Salesforce/blip2-opt-2.7b"  # You can use a different BLIP-2 model variant
blip_model = Blip2Model.from_pretrained(model_id).to(device)
processor = Blip2Processor.from_pretrained(model_id)

# Define custom dataset
class UIDataset(Dataset):
    def __init__(self, root_dir, processor=None):
        self.root_dir = root_dir
        self.processor = processor
        self.image_paths = []

        for label in os.listdir(self.root_dir):
            subdir_path = os.path.join(self.root_dir, label)
            if os.path.isdir(subdir_path):
                for filename in os.listdir(subdir_path):
                    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff', '.webp')):
                        file_path = os.path.join(subdir_path, filename)
                        self.image_paths.append((file_path, label))  # Store (image path, label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        file_path, label = self.image_paths[index]

        image = Image.open(file_path).convert("RGB")  

        if self.processor:
            inputs = self.processor(images=image, return_tensors="pt")
            image_tensor = inputs["pixel_values"].squeeze(0)  # Extract processed image tensor

        return image_tensor, label


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/122k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_dataset'

In [2]:
# Create dataset and dataloader
dataset_path = "/Users/yuriivoievidka/UCU/CV/cv-project/data"
ui_dataset_all = UIDataset(root_dir=dataset_path, processor=processor)
ui_dataset_all_dataloader = DataLoader(ui_dataset_all, batch_size=32, shuffle=False)

# Extract image embeddings using BLIP-2 vision encoder
with torch.no_grad():
    all_embeddings = []
    for batch in tqdm(ui_dataset_all_dataloader):
        images, labels = batch
        images = images.to(device)

        embeddings = blip_model.vision_model(images)  # Extract features from BLIP-2 vision encoder
        embeddings = embeddings.pooler_output  # Take the pooled output if needed

        embeddings /= embeddings.norm(dim=-1, keepdim=True)  # Normalize embeddings

        all_embeddings.append(embeddings.cpu())

    all_embeddings = torch.cat(all_embeddings, dim=0)

print("Embeddings shape:", all_embeddings.shape)

# Convert embeddings to numpy
all_embeddings_np = all_embeddings.numpy()

# Encode labels as integers
label_to_index = {label: idx for idx, label in enumerate(set(label for _, label in ui_dataset_all.image_paths))}
all_labels_np = np.array([label_to_index[label] for _, label in ui_dataset_all.image_paths])

# Save embeddings and labels
np.save("blip2_embeddings.npy", all_embeddings_np)
np.save("blip2_labels.npy", all_labels_np)

 45%|████▍     | 58/130 [03:46<05:02,  4.20s/it]/Users/yuriivoievidka/.pyenv/versions/3.12.7/envs/torch/lib/python3.12/site-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 130/130 [07:54<00:00,  3.65s/it]

Embeddings shape: torch.Size([4149, 1408])


In [6]:
embeddings = np.load("blip2_embeddings.npy")  
labels = np.load("blip2_labels.npy")  

X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.3, random_state=42)

In [7]:
sweep_config = {
    "method": "random", 
    "metric": {
      "name": "accuracy",
      "goal": "maximize"   
    },
    "parameters": {
        "booster": {
            "values": ["gbtree","gblinear"]
        },
        "max_depth": {
            "values": [3, 6, 9, 12]
        },
        "learning_rate": {
            "values": [0.1, 0.05, 0.2]
        },
        "subsample": {
            "values": [1, 0.5, 0.3]
        }
    }
}

In [10]:
sweep_id = wandb.sweep(sweep_config, project="ui-classification-experiments")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: fjdy11wf
Sweep URL: https://wandb.ai/urik-voevidka-ukrainian-catholic-university/ui-classification-experiments/sweeps/fjdy11wf


In [11]:
def train():
  config={
        "learning_rate": 0.02,
        "architecture": "XGBoost",
        "dataset": "DesktopUI",
        "epochs": 100,
        "model": "XGBoost",
        "test_size": 0.3,
        "random_state": 42
    }

  wandb.init(config=config)  # defaults are over-ridden during the sweep
  config = wandb.config

  model = xgb.XGBClassifier(objective="multi:softmax", booster=config.booster, max_depth=config.max_depth,
                        learning_rate=config.learning_rate, subsample=config.subsample)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)
  predictions = [round(value) for value in y_pred]

  accuracy = accuracy_score(y_test, predictions)
  
  f1 = f1_score(y_test, predictions, average="weighted")
  recall = recall_score(y_test, predictions, average="weighted")
  precision = precision_score(y_test, predictions, average="weighted")

  wandb.log({
      "accuracy": accuracy,
      "f1_score": f1,
      "recall": recall, 
      "precision": precision,
      "classification_report": classification_report(y_test, predictions, output_dict=True)
  })
  

In [12]:
wandb.agent(sweep_id, train, count=25)

wandb: Agent Starting Run: c6k7o7eg with config:
wandb: 	booster: gblinear
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 6
wandb: 	subsample: 0.3
wandb: Currently logged in as: urik-voevidka (urik-voevidka-ukrainian-catholic-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[15:54:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "max_depth", "subsample" } are not used.



accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.84578
f1_score,0.84659
precision,0.84891
recall,0.84578


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: h3aravin with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.05
wandb: 	max_depth: 3
wandb: 	subsample: 1


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.78474
f1_score,0.77823
precision,0.78898
recall,0.78474


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: rh7ibcw3 with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 12
wandb: 	subsample: 0.5


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.82972
f1_score,0.82819
precision,0.83058
recall,0.82972


wandb: Agent Starting Run: do5fhng0 with config:
wandb: 	booster: gblinear
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 12
wandb: 	subsample: 1


[16:00:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "max_depth", "subsample" } are not used.



accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.84659
f1_score,0.8474
precision,0.84967
recall,0.84659
